# Выбор локации для скважины

Мы работаем в добывающей компании. Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных

Нам известно содержимое столбцов
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
# Импортирование необходимых модулей и атрибутов
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [2]:
# Создадим вспомогательную переменную
seed = 12345

# Сперва объявим функцию, которая будет читать файлы
def pth_load(pth1, pth2):
    """Sapport using os.path.exists. Load local file in primarily

    :param pth1: local addres of file
    :type pth1: object
    :param pth2: external addres of file
    :type pth2: object
    
    :raises ValueError: if file not found in addresses
    
    :rtype: DataFrame
    :return: foundly file in the form of DataFrame
    """
    if os.path.exists(pth1):
        df = pd.read_csv(pth1)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2)
    else:
        print('Something is wrong')
    return df

# Прочитаем файлы 'geo_data_0.csv', 'geo_data_1.csv', 'geo_data_2.csv' и сохраним под именами 'df1', 'df2', 'df3' соответственно
df1 = pth_load('geo_data_0.csv', '/datasets/geo_data_0.csv')
df2 = pth_load('geo_data_1.csv', '/datasets/geo_data_1.csv')
df3 = pth_load('geo_data_2.csv', '/datasets/geo_data_2.csv')

# Для получения имформации о наборе данных df1: 
# Вызовем метод 'info()' и напечатаем пять случайных строк таблицы
df1.info()
df1.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
27054,EjLhx,0.395732,-0.081706,-0.762533,52.572225
88457,YP6Ri,0.081173,0.084802,4.442824,80.881954
24573,yRlq8,1.273738,-0.357244,-0.493662,120.842836
33252,VZJTf,1.596529,-0.554906,6.332129,180.215051
62576,xJA8y,2.008739,0.352234,4.227893,82.682259


In [3]:
# Для получения имформации о наборе данных df2: 
# Вызовем метод 'info()' и напечатаем пять случайных строк таблицы
df2.info()
df2.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
65054,ZweBZ,4.598647,-3.254971,0.994026,26.953261
66668,w75gp,1.511757,-6.882765,0.001360,0.000000
81084,rn82N,12.165744,0.166177,5.009587,134.766305
5172,0TtOy,1.687677,-5.739440,3.003518,80.859783
74160,LBOKG,-8.239657,-7.430313,4.987702,137.945408


In [4]:
# Для получения имформации о наборе данных df3: 
# Вызовем метод 'info()' и напечатаем пять случайных строк таблицы
df3.info()
df3.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
86401,jA8Up,2.934285,0.037506,5.255416,136.760617
27291,pj6th,0.407244,-0.148564,-1.089198,137.788719
9081,l33UH,0.115712,2.979892,5.112992,133.131796
88838,PpPLR,0.032437,1.626441,-0.115014,14.413662
68452,1vVyq,-0.950141,-1.764656,2.398785,39.866669


Проверим полученные наборы данных на наличие дубликатов

In [5]:
# Напечатаем количество дубликатов в каждом наборе данных
print('Дубликатов в df1:', df1.duplicated().sum())
print('Дубликатов в df2:', df2.duplicated().sum())
print('Дубликатов в df3:', df3.duplicated().sum())

Дубликатов в df1: 0
Дубликатов в df2: 0
Дубликатов в df3: 0


Полных дубликатов нет, проверим наличие дубликатов в столбце 'id'

In [6]:
# Напечатаем количество дубликатов по столбцу 'id'
print('Дубликатов в df1:', df1['id'].duplicated().sum())

df1[df1['id'].isin(df1['id'][df1['id'].duplicated()])].sort_values('id')

Дубликатов в df1: 10


,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


In [7]:
# Напечатаем количество дубликатов по столбцу 'id'
print('Дубликатов в df2:', df2['id'].duplicated().sum())

df2[df2['id'].isin(df2['id'][df2['id'].duplicated()])].sort_values('id')

Дубликатов в df2: 4


,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


In [8]:
# Напечатаем количество дубликатов по столбцу 'id'
print('Дубликатов в df3:', df3['id'].duplicated().sum())

df3[df3['id'].isin(df3['id'][df3['id'].duplicated()])].sort_values('id')

Дубликатов в df3: 4


,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


В каждом наборе данных есть несколько строк с одинаковым ID, но другие показатели у них отличаются. Предположительно, одинаковые ID им были присвоены случайно, на самом деле это разные скважины, так что не стоит удалять эти дубликаты. Но будем иметь в виду, что ID у скважин могут совпадать.

Мы получили три набора данных, каждый из которых содержит информацию о скважинах в одном регионе.

## Обучение и проверка модели

Обучим и проверим модель для каждого региона. Целевой параметр находится в столбце 'product'

In [9]:
# Напишем функцию, которая будет принимать набор данных, обучать и проверять модель и сохранять результаты
def product_predict(df):
    """Support for rapid learning and validation of the linear regression model
    
    :param df: DataFrame is not divided into features and targets
    :type pth1: DataFrame
    
    :rtype: LinearRegression, DataFrame, float, float64
    :return: a trained model, predictions and actual values, the average value of predictions, the RMSE metric
    """
    # Отделим целевой признак. Набор с остальными признаками создадим также без столбца 'id'
    target = df['product']
    features = df.drop(['product', 'id'], axis=1)
    
    # Разделим данные признаков на выборки с помощью функции "train_test_split"
    features_train, features_valid, target_train, target_valid = \
    train_test_split(features, target, test_size=0.25, random_state=seed) 
  
    # Обучим модель 
    model = LinearRegression()
    model.fit(features_train, target_train)

    # Получим предсказания модели
    predicted_valid = pd.Series(model.predict(features_valid))

    # Индексы предсказаний скопируем из индексов валидационной выборки
    predicted_valid.index = target_valid.index
    
    # Сохраним предсказания и правильные ответы в отдельный набор данных
    df_preds_and_features = pd.DataFrame(predicted_valid)
    df_preds_and_features.columns = ['predicted']
    df_preds_and_features['valid'] = target_valid
   
    # Вычислим средний предсказанный запас продукта в регионе
    mean_predicted_product = predicted_valid.mean()
    
    # Вычислим RMSE модели
    result = mean_squared_error(target_valid, predicted_valid)**0.5 
    
    # Функция возвращает модель, набор данных с предсказаниями модели и правильными ответами,
    # средний предсказанный запас продукта и RMSE модели
    return model, df_preds_and_features, mean_predicted_product, result

In [10]:
# Вызовем функцию, сохранив переменные, которые она передает
model1, df_preds_and_features1, mean_predicted_product1, result1 = product_predict(df1)

# Проверим результат
print("Средний запас предсказанного сырья: {:.2f}".format(mean_predicted_product1))
print("RMSE модели линейной регрессии на валидационной выборке: {:.4f}".format(result1))
df_preds_and_features1.head()

Средний запас предсказанного сырья: 92.59
RMSE модели линейной регрессии на валидационной выборке: 37.5794


,predicted,valid
71751,95.894952,10.038645
80493,77.572583,114.551489
2655,77.892640,132.603635
53233,90.175134,169.072125
91141,70.510088,122.325180


In [11]:
# То же самое для остальных регионов
model2, df_preds_and_features2, mean_predicted_product2, result2 = product_predict(df2)

# Проверим результат
print("Средний запас предсказанного сырья: {:.2f}".format(mean_predicted_product2))
print("RMSE модели линейной регрессии на валидационной выборке: {:.4f}".format(result2))
df_preds_and_features2.head()

Средний запас предсказанного сырья: 68.73
RMSE модели линейной регрессии на валидационной выборке: 0.8931


,predicted,valid
71751,82.663314,80.859783
80493,54.431786,53.906522
2655,29.748760,30.132364
53233,53.552133,53.906522
91141,1.243856,0.000000


In [12]:
# Третий регион
model3, df_preds_and_features3, mean_predicted_product3, result3 = product_predict(df3)

# Проверим результат
print("Средний запас предсказанного сырья: {:.2f}".format(mean_predicted_product3))
print("RMSE модели линейной регрессии на валидационной выборке: {:.4f}".format(result3))
df_preds_and_features3.head()

Средний запас предсказанного сырья: 94.97
RMSE модели линейной регрессии на валидационной выборке: 40.0297


,predicted,valid
71751,93.599633,61.212375
80493,75.105159,41.850118
2655,90.066809,57.776581
53233,105.162375,100.053761
91141,115.303310,109.897122


По какой-то причине модель линейной регрессии работает в несколько раз точнее с выборкой второго региона, чем с двумя остальными. 

Самый большой запас сырья предсказывается в третьем регионе, на втором месте первый регион, на третьем - второй регион.

## Подготовка к расчёту прибыли

Для удобства сохраним все ключевые параметры для расчетов прибыли в отдельных переменных. Затем расчитаем достаточный объем сырья для безубыточной разработки новой скважины и сравним с полученным средним запасом в каждом регионе.

* По условиям задачи каждая тысяча баррелей приносит 450 тысяч рублей дохода
* Бюджет на разработку скважин в регионе — 10 млрд рублей
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки. Следовательно, бюджет распределяется на 200 скважин

Чистая прибыль равняется операционной прибыли (которая в свою очередь равняется: валовая прибыль (которая в свою очередь равняется: оборот минус себестоимость) минус операционные расходы), минус налоги, минус обслуживание долга.
Следовательно, чистая прибыль = оборот - себестоимость - операционные расходы - налоги - обслуживание долга.

Налог учтен в информации о доходе за тысячу баррелей. 

Себестоимость, операционные расходы, обслуживание долга нам не известны. Известен только бюджет на весь регион. 

На каждую скважину уйдет такая часть бюджета, сколько всего планируется скважин в регионе.

Следовательно, чистая прибыль = доход от тысячи баррелей * объем запасов в скважине - бюджет на разработку скважины.

Получаем формулу, объем запасов в скважине, достаточный для выхода на прибыль = бюджет на разработку скважины / доход от тысячи баррелей.

In [13]:
# INCOME_FROM_PRODUCT - доход от продукта
INCOME_FROM_PRODUCT = 450000

# DEVELOPMENT_BUDGET - бюджет на разработку скважин
DEVELOPMENT_BUDGET = 10000000000

# EXAMINED_BOREHOLES - исследуемых скважин
EXAMINED_BOREHOLES = 500

# SELECTED_BOREHOLES - выбранных скважин
SELECTED_BOREHOLES = 200

# budget_for_borehole - бюджет на скважину
budget_for_borehole = DEVELOPMENT_BUDGET / SELECTED_BOREHOLES

# Чистая прибыль, если бы мы могли ее рассчитать
# clear_profit = INCOME_FROM_PRODUCT * объем_запасов - budget_for_borehole

# enough_volume - достаточный объем запасов
enough_volume = budget_for_borehole / INCOME_FROM_PRODUCT

In [14]:
# Напечатаем результат
print("Необходимый объем запасов в скважине, тысяч баррелей: {:.2f}".format(enough_volume))

Необходимый объем запасов в скважине, тысяч баррелей: 111.11


Средний запас предсказанного сырья в третьем регионе составляет 94.97 тысяч баррелей, так что достижение порога прибыльности у многих скважин вполне вероятна.

Напишем функцию для расчета прибыли по выбранным скважинам

In [15]:
# Объявим функцию. Функция будет принимать набор данных с предсказаниями и правильными ответами, которые мы получили ранее
def profit_calculation(df):
    """Calculates the real profit of the best wells according to the forecasts of the models
    
    :param df: DataFrame with predictions and real values of the volumes of reserves in the well
    :type pth1: DataFrame
    
    :rtype: float64
    :return: the amount of profit including all expenses
    """
    # Выберим скважины с SELECTED_BOREHOLES наибольшими значениями предсказаний
    best_boreholes = df.nlargest(SELECTED_BOREHOLES, ['predicted'])
    
    # Вычислим сумму настоящих значений лучших, по мнению модели, скважин
    best_boreholes_sum = best_boreholes['valid'].sum()
      
    # Расчитаем прибыль для полученного объема сырья
    assumed_profit = INCOME_FROM_PRODUCT * best_boreholes_sum - DEVELOPMENT_BUDGET
    
    # Функция возвращает полученное значение
    return assumed_profit

In [16]:
# Испытаем функцию
profit_calculation(df_preds_and_features1)

3320826043.1398506

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона:
Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выберем регион с наибольшей средней прибылью.

In [17]:
# Cоздадим объект RandomState
state = RandomState(seed) 

# Доля выбранных скважин от разведуемых
boreholes_proportion  = SELECTED_BOREHOLES / EXAMINED_BOREHOLES

In [18]:
# Объявим новую функцию. Функция будет принимать набор данных с предсказаниями и правильными ответами
def bootstrap_exam(df):
    """With the help bootstrap technique calculates possible profit options and calculates the totals
    
    :param df: DataFrame with predictions and real values of the volumes of reserves in the well
    :type pth1: DataFrame
    
    :rtype: Series, float, float64, float64, float64
    :return: options for the final profit, average profit, upper and lower values of 95% confidence interval, probability of loss
    """
    # Создадим вспомогательный набор данных
    values = []
    
    # Формируем выборки функцией bootstrap
    for i in range(1000):
        target_subsample = df.sample(
            n=EXAMINED_BOREHOLES,
            replace=True,
            random_state=state
        )
        
        # Вычисляем и сохраняем с помощью функции profit_calculation прибыль с каждой выборки
        values.append(profit_calculation(target_subsample))
    
    # Преобразовываем в список
    values = pd.Series(values)
    
    # Найдем среднюю прибыль
    mean = values.mean()
   
    # Найдем элементы 95%-го доверительного интервала
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    # Найдем риск убытков
    probability_of_loss = (values < 0).mean()

    # Функция возвращает полученный список, среднюю прибыль, доверительный интервал, риск убытков
    return values, mean, lower, upper, probability_of_loss

In [19]:
# Вызовем функцию к первому региону и сохраним переменные
values1, mean1, lower1, upper1, probability_of_loss1 = bootstrap_exam(df_preds_and_features1)

# Напечатаем полученные переменные
print("Средняя прибыль в выборках, руб.: {:.0f}".format(mean1))
print("95%-й доверительный интервал, руб.: {:.0f}, {:.0f}".format(lower1, upper1))
print("Вероятность убытка: {:.1%}".format(probability_of_loss1))

Средняя прибыль в выборках, руб.: 396164985
95%-й доверительный интервал, руб.: -111215546, 909766942
Вероятность убытка: 6.9%


In [20]:
# Вызовем функцию ко второму региону и сохраним переменные
values2, mean2, lower2, upper2, probability_of_loss2 = bootstrap_exam(df_preds_and_features2)

# Напечатаем полученные переменные
print("Средняя прибыль в выборках, руб.: {:.0f}".format(mean2))
print("95%-й доверительный интервал, руб.: {:.0f}, {:.0f}".format(lower2, upper2))
print("Вероятность убытка: {:.1%}".format(probability_of_loss2))

Средняя прибыль в выборках, руб.: 461155817
95%-й доверительный интервал, руб.: 78050811, 862952060
Вероятность убытка: 0.7%


In [21]:
# Вызовем функцию к третьему региону и сохраним переменные
values3, mean3, lower3, upper3, probability_of_loss3 = bootstrap_exam(df_preds_and_features3)

# Напечатаем полученные переменные
print("Средняя прибыль в выборках, руб.: {:.0f}".format(mean3))
print("95%-й доверительный интервал, руб.: {:.0f}, {:.0f}".format(lower3, upper3))
print("Вероятность убытка: {:.1%}".format(probability_of_loss3))

Средняя прибыль в выборках, руб.: 392950475
95%-й доверительный интервал, руб.: -112227625, 934562915
Вероятность убытка: 6.5%


Наибольшую среднюю прибыль показал второй регион. Первый и второй регионы превысили 2.5 процентный порог шанса убытка и не попали бы в рекомендацию к разработке в любом случае.

Лучшим регионом для разработки является второй регион. Объемы запасов в предсказаниях в выборках в этом регионе самые высокие из имеющихся, а риск убытка самый низкий.